# Information content of a dynamic process

One of the major challenges is testing the utility of information theory applied to complex adaptive behavior. Can we utilize the abstraction of flow of information within a complex adaptive system to identify _how_ information traverses through the network that is coupled  to some systemic behavior. Here we are looking at meta-stability. One of the goals of the paper is how the transition between stable states occur. 

In the Ising model meta stable behavior occurs when the order parameter $T$ is below or equal to the critical temperature $T_c$. When $T < T_c$, the average phase of the spins in the system align for periods. The transition between a phase shift is at the lowest level caused through thermal noise, i.e. a node chooses its next state with some probability based on the state of its neighbors. Here, we are looking through the lense of a mechanistic description:

- How does the tipping point traverse through the system?
- What are the information properties of nodes in prior, during and after a tipping point?
    - Can the tipping point be predicted by some earlier time series by looking at the information flows within the network?


It is essential to realize that each nodes constructs an energy landscape or manifold that it is able traverse through inputs. Some points in the landscape will lead to stability whereas others are more unstable. For a node situated in a complex adaptive system, this energy landscape will vary as a function of its topological connectedness. The hope of this article will be that information theory can provide a description that identifies tipping point dynamics irregardless of a lower topological or direct energy descriptie context.

The connectedness of complex networks and the energy landscapes, gives inspiration as to what mechanism underlies the tipping point. Namely, nodes with high variability (low connectedness) will recruit nodes with lower variability (medium connectedness) and at some point convert the nodes with the lowest variability (high connected nodes). 



In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, '../')
from plexsim.models import *
from imi import infcy
from imi.utils import plotting
from plotly import graph_objects as go
import networkx as nx, numpy as np

f = lambda x, beta : 1 / (1 + np.exp(-x * beta) )
x = np.linspace(-10, 10)
e = f(x, 1)
fig = go.Figure(data = go.Scatter(x = x, y = e))
fig.show()

ImportError: cannot import name 'infcy' from 'imi' (unknown location)

_Probability as a function of input energy. Around E = 0, nodes with Ising dynamic will produce more variability. For $T < T_c$ the nodes tend to magnetize. Connectivity structures in the graph will produce a variety of behaviors._

## Information dissipation time
$$ IDL = t := \alpha I(X_t ; Y_0)$$

## Asymptotic information
Need an alternative for this one



This leads to the following hypothesis:
- $H_0$: The tipping point is not initiated by nodes with high IDL, low AI.
- $H_1$: The tipping point is iniated by nodes with high IDL, low AI


# Simulation parameters

- Look at the distance to tipping point
- Create distribution P(S|M) where M is the distance to tipping point
- Start with magnetization to zero and walk the hill
- Create decreasing degree graph


In [ ]:
from imi.utils.graph import *
from matplotlib import pyplot as plt
cmap = plt.cm.get_cmap('nipy_spectral')
idx = np.linspace(0, 256, m.nNodes, dtype = int)
colors = cmap(idx)

agp = plotting.addGraphPretty
g = recursive_tree(5, 0)
settings = dict(
graph = g, 
sampleSize = 1)
m = Potts(**settings)

fig, ax = plt.subplots( figsize = (10, 10) )
pos = plotting.get_shell_layout(m.graph, rotate = -.1)
agp(m.graph, ax, positions = pos, cmap = colors)
ax.axis('off')
fig.show()

In [ ]:

temps = np.linspace(0, 10, 30); #temps[-1] = 10000
out   = m.magnetize(temps, n = int(1e5))

In [ ]:
from scipy import optimize
def sig(x, a, b, c, d):
    return a / (1 + b * np.exp(c * (x - d)))
opts, cov = optimize.curve_fit(sig, xdata = temps, ydata = out[0],\
                      maxfev = 100_000)

thetas = [.5]

data = []

simTemps = []
# bounds = optimize.Bounds(0, np.inf)
for theta in thetas:
    res = optimize.minimize(lambda x: abs(sig(x, *opts) - theta), \
                            x0 = .1,\
#                             bounds = ((1, 0),),\
                            method = 'TNC',\
                           )
    simTemps.append(res.x) 
    line = go.Scatter(x = res.x, y = sig(res.x, *opts), \
                      mode = 'markers',\
                     marker = dict(size = 20,\
                                   color = 'red'),\
                     name = f'{theta}')
    data.append(line) 
    
xr = np.linspace(0, 10)
data.append(go.Scatter(x = temps, y = out[0], line = dict(color = 'blue')))
data.append(go.Scatter(x = xr, y = sig(xr, *opts), line = dict(color = 'green')))
fig = go.Figure(data = data)
fig.update_layout(xaxis = dict(title = 'T'),\
                 yaxis = dict(title = '<M>'))
fig.show()


# Finding the tipping points
- Simulate and use the fact that it is always bound between (0,1)
- Apply sign
- Search for diffs

In [ ]:
# get a feeling for tippings points of the system
m.t = simTemps[0] * 1.
# m.t = 1.5
# m.t = 0.5
# m.mcmc.p_recomb = .0


# print(m.get_settings()['nudge'])
# for k,v in m.get_settings().items(): print(k)
# m.states = 0
m.reset()
# m.sampleSize = m.nNodes
spacing = int(1e1)
# spacing = 1
sim = m.simulate(int(1e6))[::spacing]

In [ ]:
from scipy import ndimage, signal, fft
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.decomposition import PCA
from scipy.stats import zscore, zmap
from scipy import optimize

from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.preprocessing import quantile_transform, normalize

sigma = 10
mean = sim.mean(1)
filtered = ndimage.gaussian_filter(mean, sigma = sigma)
swaps = np.abs(np.gradient(np.sign(filtered * 2 - 1)))
# import p
# distance = 2 * sigma #(m.nNodes - m.sampleSize + 1)
# scores = np.abs(np.gradient(np.sign(filtered * 2 - 1)))
# idx = np.where(scores)[0]
lines=  [
    go.Scatter(y = mean, opacity = .3, name = "Mean magnetization"),\
    go.Scatter(y = swaps, opacity = .3),
    go.Scatter(y = filtered, opacity = .3),
    go.Scatter(y = sim[:, 0], opacity = .25, name = 'Hub'),\
    go.Scatter(y = sim[:, 1:5].mean(1), opacity = .1)
]
fig = go.Figure(data = lines)
fig.update_layout(xaxis = dict(title = 'time'), )
fig.show()

In [ ]:
from scipy import ndimage
from imi.utils.signal import find_peaks
# m.t = 1o
surround = 10
start    = 3
sigma    = 1000
# surround = 3
m.sampleSize = 1
bins = np.linspace(-1, 1, 20)
dist = find_peaks(m, 
                  5, 
                  buffer_size = int(1e6), \
                  bins        = bins,\
                  surround    = surround, \
                  start       = start,
                  target      = 0,
                  sigma       = sigma
                 )



In [ ]:
fig, ax = plt.subplots()
xr = np.arange(len(dist))
labels = np.asarray([np.round(i, 2) for i in dist.keys()])
height = np.asarray([len(i) for i in dist.values()])

ax.bar(labels, height, width = 1/(len(labels)))
ax.set(ylabel = "freq", xlabel = "distance")
fig.show()

for k, v in dist.items():
    print(round(k,2), len(v), end = '\t')

# Show MI decays as function of distance to target 

In [ ]:
from Toolbox import infcy
np.seterr('ignore')
from pyprind import prog_bar
# m.p_recomb = .0
# m.sampleSize = m.nNodes # warning
m.sampleSize = 1
SIM = infcy.Simulator(m)
mis = {}
for k, v in prog_bar(dist.items()):
    try:
        output = SIM.forward(v, repeats = int(1e2), time_steps = 10)
        snapshots, conditional = output.values()
        px, mi = infcy.mutualInformation(conditional, snapshots)
        mis[k] = mi
    #         mis.append(mi)
    except Exception as e:
        print(e)
        continue

In [ ]:
traces = []
titles = [f'$distance {{ {idx} }}$' for idx in dist]
from plotly.subplots import make_subplots
# print(len(mis))
# print(mis)

labels = [np.round(i, 2) if isinstance(i, float) else i for i in mis]
fig = make_subplots(1, len(mis), shared_yaxes = True,\
                   x_title = 'Time', y_title = 'Node',\
                   subplot_titles = labels,\
                   horizontal_spacing = .01)
for idx, (k, mi) in enumerate(mis.items()):
    z = (mi - mi.min()) / (mi.max() - mi.min())
    
    trace = go.Heatmap(z = z.T, \
                       zmin = 0, zmax = 1,\
                      colorbar = dict(title = dict( text = '$I(S_i^{t_0 +t}; s_i^t)$',\
                                                  ),\
                                     xpad = 0,\
                                     ypad=0,\
                                     xanchor = 'left'))
    fig.add_trace(trace, 1, idx + 1)
# fig.update_layout(\f
#                  margin = dict(r = 50))
fig.show()

# Show results in the network

In [ ]:
# from matplotlib import pyplot as plt
from Utils.plotting import addGraphPretty as agp

c = len(mis) // 3
r = 3
print(r, c)
%matplotlib inline
fig, ax = plt.subplots(r, c, figsize = (45, 15))
# pos = nx.kamada_kawai_layout(g, scale = 20)
def get_shells(graph):
    shells = {}
    for k, v in dict(graph.degree()).items():
        shells[v] = shells.get(v, []) + [k]
    shells = dict(sorted(((k, v) for k, v in shells.items()), key = lambda x: x[0])[::-1])
    return shells
shells = get_shells(m.graph)
pos = nx.shell_layout(m.graph,
                      nlist = list(shells.values()),\
                      rotate = 0,\
                      scale = 1,
                 )

patches = None

mis = dict(sorted(mis.items(), key = lambda x: x[0]))

labels = [np.round(i, 2) if isinstance(i, float) else i for i in mis]
for axi, tmp, l in zip(ax.flatten(), mis.values(), labels):
    patches = agp(m.graph, axi, positions = pos, 
                  mapping = m.adj.mapping,\
                 cmap = colors).patches
    radius = tmp.sum(0)
    radius = (radius - radius.min()) / (radius.max() - radius.min())
    radius[np.isnan(radius) == True] = 1
    for patch in patches:
        if label := patch.get_label():
                idx = m.adj.mapping[label]
                patch.set(radius = radius[idx] * .03)
    axi.set_title(l, fontsize = 30)
    axi.axis('off')
fig.tight_layout()
fig.subplots_adjust(wspace = 0, hspace = 0)
fig.show()

# Estimate AI, IDT

In [ ]:
from scipy import optimize
from scipy import integrate
def f(x, a, b, c, d, e, f, g):
    return a + b * np.exp(-c * (x - d)) + e * np.exp(-f * (x - g))

# def f(x, a, b, c):
#     return a + b * np.exp(- c * x)
xr = np.arange(len(next(iter(mis.values()))))

padding = np.zeros(50)
# xr = np.insert(xr, -1, np.arange(int(1e5), int(1e5) + padding.size))
coeffs = np.zeros((len(mis), m.nNodes, f.__code__.co_argcount - 1))

features = np.zeros((len(mis), m.nNodes, 3))

start = int(1e6)
xr_extend = np.arange(start, start + padding.size)
for idx, (k,mi) in enumerate(mis.items()):
    for jdx, node in enumerate(mi.T):
        xre  = np.concatenate( (xr, xr_extend), axis = 0)
        nodee = np.concatenate( (node, padding) )
        a, b = optimize.curve_fit(f, xre, nodee, 
                                  maxfev = int(1e5), bounds = (0, np.inf))
        coeffs[idx, jdx] = a
        idt = optimize.minimize(lambda x: abs(f(x, *a) - .5 * (f(0, *a) - a[0]) - a[0]), 0,\
                                method = 'COBYLA',\
#                                 bounds = (0, np.inf),\
                                options = dict(maxiter = int(1e4)))
        if not idt.success:
            print(idt)
        idt_ = idt.x
        ai       = a[0]
        imi   = integrate.quad(lambda x: f(x, *a), 0, np.inf, full_output = 1)
        if imi[1] > .1:
            imi = 0
        else:
            imi = imi[0]
        features[idx, jdx] = (ai, idt_, imi)


In [ ]:
fig, ax = plt.subplots(2, len(mis), figsize = (20, 10),\
                       sharex = 'row',\
                       sharey = 'row'
                      )
xr = np.linspace(0, 10)

# mapping = {int(i): j for i, j in m.adj.mapping.items()}
for idx, (axi, (k, mi)) in enumerate(zip(ax[0].flatten(), mis.items())):
    for jdx, (c, node) in enumerate(zip(colors, mi.T)):
        axi.plot(node, color = c)
        axi.plot(xr, f(xr, *coeffs[idx, jdx]), color = c, linestyle = 'dashed')
        
        ax[1, idx].scatter(x = features[idx, jdx, 1] + 1e-16, 
                           y = features[idx, jdx, 2] + 1e-16,\
                           color = c, s = 100)
#         ax[1, idx].set_yscale('log')
#         ax[1, idx].set_xscale("log")
    axi.set_title(round(k, 2))
ma = fig.add_subplot(211, xticks = [], yticks = [],\
               frameon = False)
ma.set_xlabel('Time',labelpad = 30, fontsize = 25)
ma.set_ylabel('$I(S^{t_0 + t}; s_i^t)$', labelpad = 30, fontsize = 30)

ma = fig.add_subplot(212, xticks = [], yticks = [],\
               frameon = False)
ma.set_xlabel('IDT', labelpad = 30, fontsize = 30)
ma.set_ylabel('IMI', labelpad = 30, fontsize = 30)
fig.subplots_adjust(wspace = 0)
fig.show()

In [ ]:
paths = {}
for node in m.graph.nodes():
    l = nx.shortest_path_length(m.graph, node, "0")
    paths[l] = paths.get(l, []) + [m.adj.mapping[node]]
# print(paths) 
lm = plotting.get_linear_cmap(len(paths))
fig, ax = plt.subplots(2, len(mis), figsize = (20, 10),\
                       sharex = 'row',\
                       sharey = 'row'
                      )
xr = np.linspace(0, 10)

for idx, (axi, (k, mi)) in enumerate(zip(ax[0].flatten(), mis.items())):
    for jdx, (c, (l, nodes)) in enumerate(zip(lm, paths.items())):
        axi.plot(mi[:, nodes].mean(1), color = c, label = l)
        
        fe = features[idx, nodes].mean(0)
        ax[1, idx].scatter(x = fe[0]+ 1e-16, 
                           y = fe[2] + 1e-16,\
                           color = c, s = 100)
        
        ax[1, idx].set_yscale('log')
#         ax[1, idx].set_xscale("log")
    axi.set_title(round(k, 2))
axi.legend()
ma = fig.add_subplot(211, xticks = [], yticks = [],\
               frameon = False)
ma.set_xlabel('Time',labelpad = 30, fontsize = 25)
ma.set_ylabel('$I(S^{t_0 + t}; s_i^t)$', labelpad = 30, fontsize = 30)

ma = fig.add_subplot(212, xticks = [], yticks = [],\
               frameon = False)
ma.set_xlabel('$\phi_0$', labelpad = 30, fontsize = 30)
ma.set_ylabel('$\phi_1$', labelpad = 30, fontsize = 30)
fig.subplots_adjust(wspace = 0)
fig.show()